In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random
from torch.utils.tensorboard import SummaryWriter


In [31]:
spacy_ger = spacy.load('de_core_news_sm')
spacy_eng = spacy.load('en_core_web_sm')

In [32]:
def tokenizer_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]

In [33]:
def tokenizer_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]

In [34]:
# FIELD FOR DEFINING PREPROCESSING

german = Field(tokenize=tokenizer_ger, lower=True, init_token='<sos>', eos_token='<eos>')


english = Field(tokenize=tokenizer_eng, lower=True, init_token='<sos>', eos_token='<eos>')

In [35]:
train_data, validation_data, test_data = Multi30k.splits(fields=(english, german),exts=('.en', '.de'))

In [36]:
german.build_vocab(train_data, max_size=10000, min_freq=1)
english.build_vocab(train_data, max_size=10000, min_freq=1)


In [37]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        

    def forward(self, x):
        # x shape :(seq_length, batch_size)

        embedding = self.dropout(self.embedding(x))
        # shape: (seq_length, batch_size, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)


        return hidden, cell

    

In [38]:
class Decoder(nn.Module):
    
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, p):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        # shape of x : (batch_size) but we want (1, batch_size)
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # shape; (1, batch_size, embedding_size)

        outputs, (hidden,cell) = self.rnn(embedding, (hidden, cell))
        # outputs: (1, batch_size, hidden_size)

        predictions = self.fc(outputs)
        # predictions: (1, batch_size, length_of_vocab)

        predictions = predictions.squeeze(0)

        return predictions, hidden, cell

In [39]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        # source: (target_len, batch_size)
        target_len = target.shape[0]
        target_vocab_size = len(german.vocab)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size)
        hidden, cell = self.encoder(source)

        # grab start token
        x = target[0]

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)

            outputs[t] = output

            # output : (batch_size, german_vocab_size)
            best_guess = output.argmax(1)

            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs


In [40]:
# TRAINING

num_epochs = 20
learning_rate = 0.001
batch_size = 64

load_model = False
input_size_encoder = len(english.vocab)
input_size_decoder = len(german.vocab)
output_size = len(german.vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 128
num_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5

# tensorboard
writer = SummaryWriter(f'runs/loss_plot')
step = 0

train_iterator, validation_iterator, test_iterator = BucketIterator.splits((train_data, validation_data, test_data),
batch_size=batch_size, sort_within_batch = True, sort_key = lambda x: len(x.src))

In [41]:
encoder_net = Encoder(input_size_encoder, encoder_embedding_size,  hidden_size, num_layers, enc_dropout)



In [42]:
decoder_net = Decoder(input_size_decoder, decoder_embedding_size, hidden_size,output_size,  num_layers, dec_dropout)


In [43]:
model = Seq2Seq(encoder_net, decoder_net)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [44]:
pad_idx = german.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

if load_model:
    load_checkpoint(torch.load('my_checkpoint.pth.ptar'), model, optimizer)

In [45]:
for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    checkpoint = {"state_dict": model.state_dict(), "optimizer": optimizer.state_dict()}
    save_checkpoint(checkpoint)



    model.train()

    for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.src
        target = batch.trg

        # Forward prop
        output = model(inp_data, target)

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin. While we're at it
        # Let's also remove the start token while we're at it
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        print(f'OUTPUT SHAPE: {output.shape}')
        print(f'TARGET SHAPE: {target.shape}')
        optimizer.zero_grad()
        loss = criterion(output, target)

        # Back prop
        loss.backward()

        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # Plot to tensorboard
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1

[Epoch 0 / 20]
=> Saving checkpoint
OUTPUT SHAPE: torch.Size([1664, 10004])
TARGET SHAPE: torch.Size([1664])
OUTPUT SHAPE: torch.Size([1216, 10004])
TARGET SHAPE: torch.Size([1216])
OUTPUT SHAPE: torch.Size([1088, 10004])
TARGET SHAPE: torch.Size([1088])
OUTPUT SHAPE: torch.Size([1280, 10004])
TARGET SHAPE: torch.Size([1280])
OUTPUT SHAPE: torch.Size([1728, 10004])
TARGET SHAPE: torch.Size([1728])
OUTPUT SHAPE: torch.Size([1856, 10004])
TARGET SHAPE: torch.Size([1856])
OUTPUT SHAPE: torch.Size([1152, 10004])
TARGET SHAPE: torch.Size([1152])
OUTPUT SHAPE: torch.Size([896, 10004])
TARGET SHAPE: torch.Size([896])
OUTPUT SHAPE: torch.Size([1216, 10004])
TARGET SHAPE: torch.Size([1216])
OUTPUT SHAPE: torch.Size([1152, 10004])
TARGET SHAPE: torch.Size([1152])
OUTPUT SHAPE: torch.Size([1600, 10004])
TARGET SHAPE: torch.Size([1600])
OUTPUT SHAPE: torch.Size([1408, 10004])
TARGET SHAPE: torch.Size([1408])
OUTPUT SHAPE: torch.Size([1152, 10004])
TARGET SHAPE: torch.Size([1152])
OUTPUT SHAPE: tor

KeyboardInterrupt: 